# 第7章 文本数据

![mindmap](https://github.com/elsieyin/joyful-pandas/blob/master/picture/c7.png)

In [39]:
import pandas as pd
import numpy as np

## 一、string类型的性质

### 1. string与object的区别
#### string类型和object不同之处有三：
#### ① 字符存取方法（string accessor methods，如str.count）会返回相应数据的Nullable类型，而object会随缺失值的存在而改变返回类型
#### ② 某些Series方法不能在string上使用，例如： Series.str.decode()，因为存储的是字符串而不是字节
#### ③ string类型在缺失值存储或运算时，类型会广播为pd.NA，而不是浮点型np.nan
#### 其余全部内容在当前版本下完全一致，但迎合Pandas的发展模式，我们仍然全部用string来操作字符串

### 2. string类型的转换
#### 如果将一个其他类型的容器直接转换string类型可能会出错：

In [2]:
#pd.Series([1,'1.']).astype('string') #报错
#pd.Series([1,2]).astype('string') #报错
#pd.Series([True,False]).astype('string') #报错

#### 当下正确的方法是分两部转换，先转为str型object，在转为string类型：

In [3]:
pd.Series([1,'1.']).astype('str').astype('string')

0     1
1    1.
dtype: string

In [4]:
pd.Series([1,2]).astype('str').astype('string')

0    1
1    2
dtype: string

In [5]:
pd.Series([True,False]).astype('str').astype('string')

0     True
1    False
dtype: string

## 二、拆分与拼接

### 1. str.split方法
#### （a）分割符与str的位置元素选取

In [6]:
s = pd.Series(['a_b_c', 'c_d_e', np.nan, 'f_g_h'], dtype="string")
s

0    a_b_c
1    c_d_e
2     <NA>
3    f_g_h
dtype: string

#### 根据某一个元素分割，默认为空格

In [7]:
s.str.split('_')

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

#### 这里需要注意split后的类型是object，因为现在Series中的元素已经不是string，而包含了list，且string类型只能含有字符串

#### 对于str方法可以进行元素的选择，如果该单元格元素是列表，那么str[i]表示取出第i个元素，如果是单个元素，则先把元素转为列表在取出

In [8]:
s.str.split('_').str[1]

0       b
1       d
2    <NA>
3       g
dtype: object

In [9]:
pd.Series(['a_b_c', ['a','b','c']], dtype="object").str[1]
#第一个元素先转为['a','_','b','_','c']

0    _
1    b
dtype: object

#### （b）其他参数
#### expand参数控制了是否将列拆开，n参数代表最多分割多少次

In [10]:
s.str.split('_',expand=True)

,0,1,2
0,a,b,c
1,c,d,e
2,<NA>,<NA>,<NA>
3,f,g,h


In [11]:
s.str.split('_',n=1)

0    [a, b_c]
1    [c, d_e]
2        <NA>
3    [f, g_h]
dtype: object

In [12]:
s.str.split('_',expand=True,n=1)

,0,1
0,a,b_c
1,c,d_e
2,<NA>,<NA>
3,f,g_h


### 2. str.cat方法
#### （a）不同对象的拼接模式
#### cat方法对于不同对象的作用结果并不相同，其中的对象包括：单列、双列、多列
#### ① 对于单个Series而言，就是指所有的元素进行字符合并为一个字符串

In [13]:
s = pd.Series(['ab',None,'d'],dtype='string')
s

0      ab
1    <NA>
2       d
dtype: string

In [14]:
s.str.cat()

'abd'

#### 其中可选sep分隔符参数，和缺失值替代字符na_rep参数

In [15]:
s.str.cat(sep=',')

'ab,d'

In [16]:
s.str.cat(sep=',',na_rep='*') # 大概是replace的意思

'ab,*,d'

#### ② 对于两个Series合并而言，是对应索引的元素进行合并

In [17]:
s2 = pd.Series(['24',None,None],dtype='string')
s2

0      24
1    <NA>
2    <NA>
dtype: string

In [18]:
s.str.cat(s2)

0    ab24
1    <NA>
2    <NA>
dtype: string

#### 同样也有相应参数，需要注意的是两个缺失值会被同时替换

In [19]:
s.str.cat(s2,sep=',',na_rep='*') # *,*

0    ab,24
1      *,*
2      d,*
dtype: string

#### ③ 多列拼接可以分为表的拼接和多Series拼接

#### 表的拼接

In [20]:
s.str.cat(pd.DataFrame({0:['1','3','5'],1:['5','b',None]},dtype='string'),na_rep='*') #不管怎样都会拼一起

0    ab15
1     *3b
2     d5*
dtype: string

#### 多个Series拼接

In [21]:
s.str.cat([s+'0'])

0    abab0
1     <NA>
2      dd0
dtype: string

In [22]:
s.str.cat([s+'0',s*2])

0    abab0abab
1         <NA>
2        dd0dd
dtype: string

#### （b）cat中的索引对齐
#### 当前版本中，如果两边合并的索引不相同且未指定join参数，默认为左连接，设置join='left'

In [23]:
s2 = pd.Series(list('abc'),index=[1,2,3],dtype='string')
s2

1    a
2    b
3    c
dtype: string

In [24]:
s.str.cat(s2,na_rep='*') # C因为index=3 rep成*了

0    ab*
1     *a
2     db
dtype: string

## 三、替换
#### 广义上的替换，就是指str.replace函数的应用，fillna是针对缺失值的替换，上一章已经提及
#### 提到替换，就不可避免地接触到正则表达式，这里默认读者已掌握常见正则表达式知识点，若对其还不了解的，可以通过[这份资料](https://regexone.com/)来熟悉

### 1. str.replace的常见用法

In [41]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'],dtype="string")
s

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

#### 第一个值写r开头的正则表达式，后一个写替换的字符串

In [42]:
s.str.replace(r'^[AB]','***')

0       ***
1       ***
2         C
3    ***aba
4    ***aca
5          
6      <NA>
7      CABA
8       dog
9       cat
dtype: string

In [43]:
s # replace不修改

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

### 2. 子组与函数替换

#### 通过正整数调用子组（0返回字符本身，从1开始才是子组）

In [27]:
s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:]+'*')

0       A
1       B
2       C
3     ba*
4     ca*
5        
6    <NA>
7     BA*
8     dog
9     cat
dtype: string

group(2)就是取出来你第二个小括号匹配的字符串
x.group(0) 返回的是单元格内的字符串本身, x.group(1)返回的是匹配到的第一个子组,   
2 是第二个子组(如果正则表达式设置了子组匹配的第二个子组)  

#### 利用?P<....>表达式可以对子组命名调用

In [28]:
s.str.replace(r'(?P<one>[ABC])(?P<two>\w+)',lambda x:x.group('two')[1:]+'*')

0       A
1       B
2       C
3     ba*
4     ca*
5        
6    <NA>
7     BA*
8     dog
9     cat
dtype: string

### 3. 关于str.replace的注意事项
#### 首先，要明确str.replace和replace并不是一个东西：
#### str.replace针对的是object类型或string类型，默认是以正则表达式为操作，目前暂时不支持DataFrame上使用
#### replace针对的是任意类型的序列或数据框，如果要以正则表达式替换，需要设置regex=True，该方法通过字典可支持多列替换
#### 但现在由于string类型的初步引入，用法上出现了一些问题，这些issue有望在以后的版本中修复
#### （a）str.replace赋值参数不得为pd.NA
#### 这听上去非常不合理，例如对满足某些正则条件的字符串替换为缺失值，直接更改为缺失值在当下版本就会报错

In [29]:
#pd.Series(['A','B'],dtype='string').str.replace(r'[A]',pd.NA) #报错
#pd.Series(['A','B'],dtype='O').str.replace(r'[A]',pd.NA) #报错

#### 此时，可以先转为object类型再转换回来，曲线救国：

In [30]:
pd.Series(['A','B'],dtype='string').astype('O').replace(r'[A]',pd.NA,regex=True).astype('string')

0    <NA>
1       B
dtype: string

#### 至于为什么不用replace函数的regex替换（但string类型replace的非正则替换是可以的），原因在下面一条

#### （b）对于string类型Series，在使用replace函数时不能使用正则表达式替换
#### 该bug现在还未修复

In [31]:
pd.Series(['A','B'],dtype='string').replace(r'[A]','C',regex=True)

0    A
1    B
dtype: string

In [32]:
pd.Series(['A','B'],dtype='O').replace(r'[A]','C',regex=True)

0    C
1    B
dtype: object

#### （c）string类型序列如果存在缺失值，不能使用replace替换

In [33]:
#pd.Series(['A',np.nan],dtype='string').replace('A','B') #报错

In [34]:
pd.Series(['A',np.nan],dtype='string').str.replace('A','B')

0       B
1    <NA>
dtype: string

#### 综上，概况的说，除非需要赋值元素为缺失值（转为object再转回来），否则请使用str.replace方法

## 四、子串匹配与提取

### 1. str.extract方法
#### （a）常见用法

In [35]:
pd.Series(['10-87', '10-88', '10-89'],dtype="string").str.extract(r'([\d]{2})-([\d]{2})')

,0,1
0,10,87
1,10,88
2,10,89


#### 使用子组名作为列名

In [36]:
pd.Series(['10-87', '10-88', '-89'],dtype="string").str.extract(r'(?P<name_1>[\d]{2})-(?P<name_2>[\d]{2})')

,name_1,name_2
0,10,87
1,10,88
2,<NA>,<NA>


#### 利用?正则标记选择部分提取

In [37]:
pd.Series(['10-87', '10-88', '-89'],dtype="string").str.extract(r'(?P<name_1>[\d]{2})?-(?P<name_2>[\d]{2})')

,name_1,name_2
0,10,87
1,10,88
2,<NA>,89


In [38]:
pd.Series(['10-87', '10-88', '10-'],dtype="string").str.extract(r'(?P<name_1>[\d]{2})-(?P<name_2>[\d]{2})?')

,name_1,name_2
0,10,87
1,10,88
2,10,<NA>


#### （b）expand参数（默认为True）

#### 对于一个子组的Series，如果expand设置为False，则返回Series，若大于一个子组，则expand参数无效，全部返回DataFrame
#### 对于一个子组的Index，如果expand设置为False，则返回提取后的Index，若大于一个子组且expand为False，报错

In [39]:
s = pd.Series(["a1", "b2", "c3"], ["A11", "B22", "C33"], dtype="string")
s.index

Index(['A11', 'B22', 'C33'], dtype='object')

In [40]:
s.str.extract(r'([\w])') # 对于一个子组的Series，expand默认为True

,0
A11,a
B22,b
C33,c


In [41]:
s.str.extract(r'([\w])',expand=False) #对于一个子组的Series，如果expand设置为False，则返回Series

A11    a
B22    b
C33    c
dtype: string

In [42]:
s.index.str.extract(r'([\w])') #对于一个子组的Index，expand默认为True

,0
0,A
1,B
2,C


In [43]:
s.index.str.extract(r'([\w])',expand=False) #对于一个子组的Index，如果expand设置为False，则返回提取后的Index

Index(['A', 'B', 'C'], dtype='object')

In [44]:
s.index.str.extract(r'([\w])([\d])')  #对于一个子组的Index，expand默认为True，取了两列

,0,1
0,A,1
1,B,2
2,C,3


In [45]:
#s.index.str.extract(r'([\w])([\d])',expand=False) #报错  大于一个子组且expand为False，报错

### 2. str.extractall方法

#### 与extract只匹配第一个符合条件的表达式不同，extractall会找出所有符合条件的字符串，并建立多级索引（即使只找到一个）

In [46]:
s = pd.Series(["a1a2", "b1", "c1"], index=["A", "B", "C"],dtype="string")
two_groups = '(?P<letter>[a-z])(?P<digit>[0-9])'
s.str.extract(two_groups, expand=True)

,letter,digit
A,a,1
B,b,1
C,c,1


In [47]:
s.str.extractall(two_groups)

letter digit
  match             
A 0          a     1
  1          a     2
B 0          b     1
C 0          c     1

In [48]:
s['A']='a1'
s.str.extractall(two_groups)

,,letter,digit
,match,,
A,0,a,1
B,0,b,1
C,0,c,1


#### 如果想查看第i层匹配，可使用xs方法

In [49]:
s = pd.Series(["a1a2", "b1b2", "c1c2"], index=["A", "B", "C"],dtype="string")
s.str.extractall(two_groups).xs(1,level='match')

,letter,digit
A,a,2
B,b,2
C,c,2


### 3. str.contains和str.match
#### 前者的作用为检测是否包含某种正则模式

In [50]:
pd.Series(['1', None, '3a', '3b', '03c'], dtype="string").str.contains(r'[0-9][a-z]')

0    False
1     <NA>
2     True
3     True
4     True
dtype: boolean

#### 可选参数为na

In [51]:
pd.Series(['1', None, '3a', '3b', '03c'], dtype="string").str.contains('a', na=False) #contains 有

0    False
1    False
2     True
3    False
4    False
dtype: boolean

#### str.match与其区别在于，match依赖于python的re.match，检测内容为是否从头开始包含该正则模式

In [52]:
pd.Series(['1', None, '3a_', '3b', '03c'], dtype="string").str.match(r'[0-9][a-z]',na=False) # match 严格

0    False
1    False
2     True
3     True
4    False
dtype: boolean

In [53]:
pd.Series(['1', None, '_3a', '3b', '03c'], dtype="string").str.match(r'[0-9][a-z]',na=False)

0    False
1    False
2    False
3     True
4    False
dtype: boolean

## 五、常用字符串方法

### 1. 过滤型方法
#### （a）str.strip
#### 常用于过滤空格

In [54]:
pd.Series(list('abc'),index=[' space1  ','space2  ','  space3'],dtype="string").index.str.strip()

Index(['space1', 'space2', 'space3'], dtype='object')

#### （b）str.lower和str.upper

In [55]:
pd.Series('A',dtype="string").str.lower()

0    a
dtype: string

In [56]:
pd.Series('a',dtype="string").str.upper()

0    A
dtype: string

#### （c）str.swapcase和str.capitalize
#### 分别表示交换字母大小写和大写首字母

In [57]:
pd.Series('abCD',dtype="string").str.swapcase()

0    ABcd
dtype: string

In [58]:
pd.Series('abCD',dtype="string").str.capitalize()

0    Abcd
dtype: string

### 2. isnumeric方法
#### 检查每一位是否都是数字，请问如何判断是否是数值？（问题二）

In [59]:
pd.Series(['1.2','1','-0.3','a',np.nan],dtype="string").str.isnumeric() #.是字符，检查是否每一位都是数字。
# Series.str.isnumeric : Check whether all characters are numeric.
# https://github.com/pandas-dev/pandas/blob/v1.0.5/pandas/core/strings.py
# 'isnumeric' L3367 3550

0    False
1     True
2    False
3    False
4     <NA>
dtype: boolean

## 六、问题与练习
### 1. 问题

#### 【问题一】 str对象方法和df/Series对象方法有什么区别？

- str.replace针对的是object类型或string类型，默认是以正则表达式为操作，目前暂时不支持DataFrame上使用
- replace针对的是任意类型的序列或数据框，如果要以正则表达式替换，需要设置regex=True，该方法通过字典可支持多列替换

#### 【问题二】 给出一列string类型，如何判断单元格是否是数值型数据？


`str.contains(r'-?([0-9]\d*)(\.\d+)?$')`

#### 【问题三】 rsplit方法的作用是什么？它在什么场合下适用？
方法通过指定分隔符对字符串进行分割并返回一个列表

#### 【问题四】 在本章的第二到第四节分别介绍了字符串类型的5类操作，请思考它们各自应用于什么场景？
- **拆分**:  某个字符串series里有相同的分隔符的时候，可以用split来分开。
- **拼接**：理解为拆分的反操作，想用某个符号作为分隔符将几个字符串连起来的时候。
- **替换**：在数据清洗的时候作用很大，可以使用正则表达式去除某些不需要的字符，数字之类的。
- **字串匹配**：作用同下，但是不用提取出来，只是看里面有没有。contains是看有没有。
- **字串提取**: 当字符串series有一定的规律的时候，可以用来提取某一部分，例如年月日可以提取年，就是提取自己需要的模式。extract是提取。

### 2. 练习
#### 【练习一】 现有一份关于字符串的数据集，请解决以下问题：
#### （a）现对字符串编码存储人员信息（在编号后添加ID列），使用如下格式：“×××（名字）：×国人，性别×，生于×年×月×日”
#### （b）将（a）中的人员生日信息部分修改为用中文表示（如一九七四年十月二十三日），其余返回格式不变。
#### （c）将（b）中的ID列结果拆分为原列表相应的5列，并使用equals检验是否一致。

In [27]:
df = pd.read_csv('data/String_data_one.csv',index_col='人员编号').astype('str')
df.head()

,姓名,国籍,性别,出生年,出生月,出生日
人员编号,,,,,,
1,aesfd,2,男,1942,8,10
2,fasefa,5,女,1985,10,4
3,aeagd,4,女,1946,10,15
4,aef,4,男,1999,5,13
5,eaf,1,女,2010,6,24


In [28]:
(df['姓名']+':'+df['国籍']+'国人，性别'
          +df['性别']+'，生于'
          +df['出生年']+'年'
          +df['出生月']+'月'+df['出生日']+'日').to_frame().rename(columns={0:'ID'}).head()

,ID
人员编号,
1,aesfd:2国人，性别男，生于1942年8月10日
2,fasefa:5国人，性别女，生于1985年10月4日
3,aeagd:4国人，性别女，生于1946年10月15日
4,aef:4国人，性别男，生于1999年5月13日
5,eaf:1国人，性别女，生于2010年6月24日


#### 【练习二】 现有一份半虚拟的数据集，第一列包含了新型冠状病毒的一些新闻标题，请解决以下问题：
#### （a）选出所有关于北京市和上海市新闻标题的所在行。
#### （b）求col2的均值。
#### （c）求col3的均值。

In [61]:
pd.read_csv('data/String_data_two.csv').head()

,col1,col2,col3
0,鄂尔多斯市第2例确诊患者治愈出院,19,363.6923
1,云南新增2例，累计124例,-67,-152.281
2,武汉协和医院14名感染医护出院,-86,325.6221
3,山东新增9例，累计307例,-74,-204.9313
4,上海开学日期延至3月,-95,4.05


#### (a）

In [33]:
df=pd.read_csv('data/String_data_two.csv')
df[df['col1'].str.contains('北京|上海')]

,col1,col2,col3
4,上海开学日期延至3月,-95,4.05
5,北京新增25例确诊病例，累计确诊253例,-4,-289.1719
6,上海新增10例，累计243例,2,-73.7105
36,上海新增14例累计233例,-55,-83
40,上海新增14例累计233例,-88,-99
53,上海7个月大婴儿确诊,-71,-391.2325
56,正直播！北京市疫情防控工作新闻发布会,-16,167.6711
62,北京新增 16 例累计确诊 228 例,-10,-367.7424
67,北京新增 16 例累计确诊 228 例,-18,-177.3911
69,正直播！北京市疫情防控工作新闻发布会,-92,107.7307


In [31]:
#df['col2'].mean() #报错
df['col2'][~(df['col2'].str.replace(r'-?\d+','True')=='True')] #这三行有问题

309    0-
396    9`
485    /7
Name: col2, dtype: object

In [32]:
df.loc[[309,396,485],'col2'] = [0,9,7]
df['col2'].astype('int').mean()

-0.984

####  （c)

In [34]:
#df['col3'].mean() #报错
#df['col3'] #报错
df.columns

Index(['col1', 'col2', 'col3  '], dtype='object')

In [35]:
df.columns = df.columns.str.strip()
df.columns

Index(['col1', 'col2', 'col3'], dtype='object')

In [36]:
df['col3'].head()

0     363.6923
1     -152.281
2     325.6221
3    -204.9313
4         4.05
Name: col3, dtype: object

In [37]:
#df['col3'].mean() #报错
df['col3'][~(df['col3'].str.replace(r'-?\d+\.?\d+','True')=='True')]

28      355`.3567
37             -5
73              1
122    9056.\2253
332    3534.6554{
370             7
Name: col3, dtype: object

- 斜杠一点是匹配一点作为小数点，斜杠是转义
- ？是非贪婪模式，匹配0个或1个， -？的意思就是有负号就匹配负号，没有就不匹配
- \d是匹配0-9，+是贪婪模式，有多少匹配多少
- + 一个或多个重复
- \.？就是匹配一个或零个点
- 这个表达式可能有漏洞，最前面加个^，最后面加个$可以分别限定开始和结束字符

In [38]:
df.loc[[28,122,332],'col3'] = [355.3567,9056.2253, 3534.6554]
df['col3'].astype('float').mean()

24.707485